In [1]:
import cupy as cp
from pyqcu.cuda import define
from pyqcu.cuda import io
from pyqcu.cuda import qcu
from pyqcu.cuda.set import params, argv, set_ptrs
#############################
params[define._LAT_X_] = 32
params[define._LAT_Y_] = 32
params[define._LAT_Z_] = 32
params[define._LAT_T_] = 32
params[define._LAT_XYZT_] = params[define._LAT_X_] * \
    params[define._LAT_Y_]*params[define._LAT_Z_]*params[define._LAT_T_]
params[define._GRID_X_], params[define._GRID_Y_], params[define._GRID_Z_], params[
    define._GRID_T_] = define.split_into_four_factors(define.size)
params[define._DATA_TYPE_] = define._LAT_C64_
params[define._SET_PLAN_] = 1
print("Parameters:", params)
argv[define._MASS_] = 0.0
argv = argv.astype(define.dtype_half(params[define._DATA_TYPE_]))
print("Arguments:", argv)
#############################
gauge_filename = f"quda_wilson-bistabcg-gauge_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5"
print("Gauge filename:", gauge_filename)
gauge = io.hdf5_xxxtzyx2grid_xxxtzyx(params, gauge_filename)
fermion_in_filename = gauge_filename.replace("gauge", "fermion-in")
print("Fermion in filename:", fermion_in_filename)
fermion_in = io.hdf5_xxxtzyx2grid_xxxtzyx(params, fermion_in_filename)
fermion_out_filename = gauge_filename.replace("gauge", "fermion-out")
print("Fermion out filename:", fermion_out_filename)
quda_fermion_out = io.hdf5_xxxtzyx2grid_xxxtzyx(params, fermion_out_filename)


@My Rank:0/1, Local Rank:0@

Parameters: [     32      32      32      32 1048576       1       1       1       1
       0       0       1       0    1000       3       0       1       4
       4       4       8      24       1      42]
Arguments: [0.e+00 1.e-09 1.e-01]
Gauge filename: quda_wilson-bistabcg-gauge_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 16
All Dest Shape: (3, 3, 4, 2, 32, 32, 32, 16)
Dest Shape: (3, 3, 4, 2, 32, 32, 32, 16)
Fermion in filename: quda_wilson-bistabcg-fermion-in_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 16
All Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Fermion out filename: quda_wilson-bistabcg-fermion-out_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Inde

In [2]:

# #############################
# fermion_out = cp.zeros_like(fermion_in)
# print("Fermion out data:", fermion_out.data)
# print("Fermion out shape:", fermion_out.shape)
# #############################
# qcu.applyInitQcu(set_ptrs, params, argv)
# for i in range(10):
#     qcu.applyWilsonBistabCgQcu(
#     fermion_out, fermion_in, gauge, set_ptrs, params)
# qcu.applyEndQcu(set_ptrs, params)
# #############################
# print("Fermion out data:", fermion_out.data)
# print("Fermion out shape:", fermion_out.shape)
# print("QUDA Fermion out data:", quda_fermion_out.data)
# print("QUDA Fermion out shape:", quda_fermion_out.shape)
# print("Difference:", cp.linalg.norm(fermion_out -
#       quda_fermion_out)/cp.linalg.norm(quda_fermion_out))
# #############################

In [3]:
import torch
from pyqcu.ascend import dslash
from pyqcu.ascend.define import *
latt_size = (32, 32, 32, 32)
mass = 0.0
kappa = 1 / (2 * mass + 8)
dtype = torch.complex64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
U = torch.from_numpy(io.pxxxtzyx2xxxtzyx(
    input_array=io.ccdptzyx2pccdtzyx(gauge=gauge)).get()).to(dtype=dtype, device=device)
b = torch.from_numpy(io.pxxxtzyx2xxxtzyx(
    input_array=fermion_in).get()).to(dtype=dtype, device=device)
refer_x = torch.from_numpy(io.pxxxtzyx2xxxtzyx(
    input_array=quda_fermion_out).get()).to(dtype=dtype, device=device)
print(f"Using device: {device}")
wilson = dslash.wilson_mg(
    latt_size=latt_size,
    kappa=kappa,
    dtype=dtype,
    device=device,
    verbose=False
)
clover = dslash.clover(
    latt_size=latt_size,
    kappa=kappa,
    dtype=dtype,
    device=device,
    verbose=False
)
wilson.check_su3(U)
clover_term = clover.make_clover(U=U)
clover_term = torch.zeros_like(clover_term) # just for test, just wilson
verbose = True
tol = 1e-9
def matvec(src: torch.Tensor, U: torch.Tensor = U, clover_term: torch.Tensor = clover_term) -> torch.Tensor:
    return wilson.give_wilson(src, U)+clover.give_clover(clover_term=clover_term, src=src)

Restored Array Shape: (3, 3, 4, 32, 32, 32, 32)
Restored Array Shape: (4, 3, 32, 32, 32, 32)
Restored Array Shape: (4, 3, 32, 32, 32, 32)
Using device: cuda


In [ ]:
torch.norm(b-matvec(src=refer_x))/torch.norm(b)

In [ ]:
hopping_plus_list = []
hopping_minus_list = []
for ward in range(4):
    hopping_plus_list.append(wilson.give_hopping_plus(ward=ward, U=U))
    hopping_minus_list.append(wilson.give_hopping_minus(ward=ward, U=U))
Ax = wilson.give_wilson(src=refer_x, U=U, with_I=True)
_Ax = refer_x.clone()
for ward in range(4):
    _Ax += wilson.give_wilson_plus(ward=ward, src=refer_x.reshape(
        [12]+list(U.shape[-4:])), hopping=hopping_plus_list[ward]).reshape([4, 3]+list(U.shape[-4:]))
    _Ax += wilson.give_wilson_minus(ward=ward, src=refer_x.reshape(
        [12]+list(U.shape[-4:])), hopping=hopping_minus_list[ward]).reshape([4, 3]+list(U.shape[-4:]))
print(torch.norm(Ax-_Ax).item()/torch.norm(Ax).item())

In [ ]:
torch.norm(b-Ax)/torch.norm(b)

In [ ]:
torch.norm(b-_Ax)/torch.norm(b)

In [ ]:
# from pyqcu.ascend import qcu
# import torch
# lat_x, lat_y, lat_z, lat_t = 32, 32, 32, 32
# _qcu = qcu(lat_size=[lat_x, lat_y, lat_z, lat_t], U=U.cpu(), dtype=torch.complex64, device=torch.device(
#     'cpu'), dslash='wilson', solver='bistabcg', verbose=True)
# _qcu.b = b.cpu()
# _qcu.x = refer_x.cpu()
# _qcu.x0 = torch.zeros_like(refer_x.cpu())
# _qcu.save(file_name='quda_wilson_inverse_-32-32-32-32-f')
# # _qcu.load()
# _qcu.init()
# _qcu.solve()
# _qcu.test()
# _qcu.mg.plot()
# _qcu.save()

give_if_multi(): False
self.lat_size: [32, 32, 32, 32]
self.grid_size: [1, 1, 1, 1]
self.grid_index: [0, 0, 0, 0]
self.dtype_list:[torch.complex64, torch.complex64, torch.complex64, torch.complex64, torch.complex64]
self.device_list:[device(type='cpu'), device(type='cpu'), device(type='cpu'), device(type='cpu'), device(type='cpu')]
self.local_lat_size: [32, 32, 32, 32]
@My Rank:0/1, Local Rank:0@

rank 0: Data is saved to quda_wilson_inverse_-32-32-32-32-f-b.h5
rank 0: Data is saved to quda_wilson_inverse_-32-32-32-32-f-x.h5
rank 0: Data is saved to quda_wilson_inverse_-32-32-32-32-f-x0.h5
rank 0: Data is saved to quda_wilson_inverse_-32-32-32-32-f-U.h5
Error: 'NoneType' object has no attribute 'cpu'
Error: 'NoneType' object has no attribute 'shape'
Error: 'NoneType' object has no attribute 'shape'
self.dof_list:[12, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24]
self.dtype_list:[torch.complex64, torch.complex64, torch.complex64, torch.complex64, torch.complex64]
self.device_list:[device(type

In [ ]:
# from pyqcu.ascend import qcu
# import torch
# lat_x, lat_y, lat_z, lat_t = 32, 32, 32, 32
# _qcu = qcu(lat_size=[lat_x, lat_y, lat_z, lat_t], dtype=torch.complex64, device=torch.device(
#     'cuda'), dslash='wilson', solver='bistabcg', verbose=True)
# _qcu.load(file_name='quda_wilson_inverse_-32-32-32-32-f')
# _qcu.init()
# _qcu.solve()
# _qcu.test()
# _qcu.mg.plot()
# _qcu.save(file_name='quda_wilson_inverse_-32-32-32-32-f')